**Neural machine Translation (english to french) using the attention mechanism**

**Import the libraries**

In [1]:
import string
import re
from pickle import dump
from unicodedata import normalize
import numpy as np
from numpy import array
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Bidirectional, Embedding,Concatenate, Permute, Dot, Input, LSTM, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras.models import load_model, Model

Load dataset and preprocess it (delete ponctuation and special characters,convert to lowercase)

In [2]:
def load_document(filename):
    file = open(filename, mode='r', encoding='utf-8')
    text = file.read()
    file.close()
    return text

def split_text_to_pairs(text):
    lines = text.strip().split('\n')
    pairs = []
    for line in lines:
        index = line.find('CC-BY')
        new_line = line[:index-1]
        eng,fr = new_line.split('\t')
        pairs.append((eng,fr))
    return pairs

def preprocess(lines):
    preprocessed = list()
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            line = line.split()
            cleaned_sentence = ''
            for word in line :
                word = word.lower()
                word = re.sub(r'([!.?])', r' \1', word)
                word = re.sub(r'[^a-zA-Z.!?]+', r' ', word)
                word = word.translate(table)
                word = re.sub(r'\s+', r' ', word)
                cleaned_sentence += " " + word
            clean_pair.append(''.join(cleaned_sentence))
        preprocessed.append(clean_pair)
    return array(preprocessed)


def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)

In [3]:
filename = '../input/fraeng/fra.txt'
doc = load_document(filename)
pairs = split_text_to_pairs(doc)
clean_pairs = preprocess(pairs)
save_clean_data(clean_pairs, 'english-french.pkl')
for i in range(100):
	print('[%s] => [%s]' % (clean_pairs[i,0], clean_pairs[i,1]))

In [4]:
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))
 
def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)
raw_dataset = load_clean_sentences('english-french.pkl')
n_sentences = 15500
dataset = raw_dataset[:n_sentences, :]
shuffle(dataset)
train = dataset
test = dataset[15400:]
save_clean_data(dataset, 'english-french-both.pkl')
save_clean_data(train, 'english-french-train.pkl')
save_clean_data(test, 'english-french-test.pkl')

In [5]:
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))
 
dataset = load_clean_sentences('english-french-both.pkl')
train = load_clean_sentences('english-french-train.pkl')
test = load_clean_sentences('english-french-test.pkl')

In [6]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
def max_length(lines):
    return max(len(line.split()) for line in lines)

eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
fr_tokenizer = create_tokenizer(dataset[:, 1])
fr_vocab_size = len(fr_tokenizer.word_index) + 1
fr_length = max_length(dataset[:, 1])
print('French Vocabulary Size: %d' % fr_vocab_size)
print('french Max Length: %d' % (fr_length))

tokenize the text and encode the output with one hot encoding

In [7]:
def encode_sequences(tokenizer, length, lines):
    X = tokenizer.texts_to_sequences(lines)
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

def encode_output(sequences, vocab_size):
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

In [8]:
trainX = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = encode_sequences(fr_tokenizer, fr_length, train[:, 1])
trainY = encode_output(trainY, fr_vocab_size)
testX = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = encode_sequences(fr_tokenizer, fr_length, test[:, 1])
testY = encode_output(testY, fr_vocab_size)

create the attention model

In [9]:
repeator = RepeatVector(eng_length)
concatenator = Concatenate(axis=-1)
densor1 = Dense(128, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation('softmax', name='attention_weights') 
dotor = Dot(axes = 1)

In [10]:
def one_step_attention(a, s_prev):
    s_prev = repeator(s_prev)
    concat = concatenator([a,s_prev])
    e = densor1(concat)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas,a])
    return context

In [11]:
n_a = 128 
n_s = 256 
post_activation_LSTM_cell = LSTM(n_s, return_state = True) 
output_layer = Dense(fr_vocab_size, activation='softmax')

In [12]:
def modelf(Tx, Ty, n_a, n_s, input_vocab_size, output_vocab_size):
    X = Input(shape=(Tx,))
    X1 = Embedding(input_vocab_size, 512, mask_zero=True)(X)
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    outputs = []
    a = Bidirectional(LSTM(units=n_a, return_sequences=True))(X1)
    for t in range(Ty):
        context = one_step_attention(a, s)
        s, _, c = next_hidden_state, _ , next_cell_state = post_activation_LSTM_cell(inputs=context, initial_state=[s, c])
        out =output_layer(s)
        outputs.append(out)
    outputs = tf.stack(outputs,axis=1)
    model = Model(inputs=[X,s0,c0],outputs=outputs)
    return model

In [13]:
model = modelf(eng_length,fr_length,n_a,n_s,eng_vocab_size, fr_vocab_size)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True)

Train the model for 30 epochs

In [14]:
s0 = np.zeros((trainX.shape[0], n_s))
c0 = np.zeros((trainX.shape[0], n_s))
checkpoint = ModelCheckpoint('best_weights.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit([trainX,s0,c0], trainY, epochs=30, batch_size=64, validation_split=0.1, callbacks=[checkpoint], verbose=2)

make predictions on the test dataset

In [15]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [16]:
def predict_sequence(model, tokenizer, source):
    s00 = np.zeros((1, n_s))
    c00 = np.zeros((1, n_s))
    prediction = model.predict([source,s00,c00], verbose=0)[0]
    integers = [argmax(vector) for vector in prediction]
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

In [17]:
def evaluate_model(model, tokenizer, sources, raw_dataset):
    actual, predicted = list(), list()
    for i, source in enumerate(sources):
        source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, fr_tokenizer, source)
        raw_src, raw_target = raw_dataset[i]
        print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
        actual.append([raw_target.split()])
        predicted.append(translation.split())

In [18]:
evaluate_model(model, fr_tokenizer, testX, test)